In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers
import matplotlib.pyplot as plt 
from keras import Sequential
import numpy as np
import pandas as pd

Ќе илустрираме пример за користење на Ограничени Болцманови машини за колаборативно филтрирање со цел да се развие систем за препораки на филмови.

In [3]:
!wget -c https://raw.githubusercontent.com/fawazsiddiqi/recommendation-system-with-a-Restricted-Boltzmann-Machine-using-tensorflow/master/data/ml-1m.zip -O moviedataset.zip
!unzip -o moviedataset.zip

--2025-02-09 23:24:28--  https://raw.githubusercontent.com/fawazsiddiqi/recommendation-system-with-a-Restricted-Boltzmann-Machine-using-tensorflow/master/data/ml-1m.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  moviedataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


Потоа го вчитуваме податочното множество при што наведуваме дека како сепаратор треба да се користи ‘::’. За параметарот header проследуваме вредност None бидејќи во фајловите во податочното множество нема заглавје (header) на колоните.


In [13]:
movies_df = pd.read_csv('ml-1m/movies.dat', sep='::',header=None, engine='python', encoding='ISO-8859-1')
movies_df.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
movies_df.columns = ['MovieID', 'Movie', 'Genre']
movies_df.head()

,MovieID,Movie,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings_df = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None, engine="python")
ratings_df.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [20]:
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [17]:
users_df = pd.read_csv('ml-1m/users.dat', header=None, sep="::", engine="python")
users_df.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Потоа правиме нормализација на вредностите на оцените така што ги доведуваме да бидат вредности во интервалот [0,1].


In [22]:
user_rating_df = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating') # pravime PivotTable za sekoj MovieID i gi vnesuvame ratings za toj film od 'Ratings'
norm_user_rating_df = user_rating_df.fillna(0) / 5.0
trX = norm_user_rating_df.values

# Поставување на параметрите и креирање на мрежата

Бројот на неврони во скриениот слој го поставуваме на 20.

A tensor is a mathematical object that generalizes scalars, vectors, and matrices to higher dimensions. In the context of machine learning and deep learning, tensors are multi-dimensional arrays used to store data. They are a fundamental data structure in libraries like TensorFlow and PyTorch

Initializes the parameters for a Restricted Boltzmann Machine (RBM) used in a collaborative filtering system for movie recommendations.

A Boltzmann Machine is a type of stochastic recurrent neural network that can learn a probability distribution over its set of inputs. It is named after the Boltzmann distribution in statistical mechanics. There are two main types of Boltzmann Machines: the standard Boltzmann Machine and the Restricted Boltzmann Machine (RBM).  

Key Concepts:
Neurons: The network consists of neurons that are either in a visible layer (input layer) or a hidden layer.

Weights: Connections between neurons have weights that are adjusted during training.

Energy Function: The network has an energy function that measures the "harmony" of a particular state of the network.

Training: The goal of training is to adjust the weights to minimize the energy of the network for the training data.

In [62]:
#broj na vlezni nevroni
visibleUnits = len(user_rating_df.columns) #The number of visible units (input neurons) is set to the number of columns in user_rating_df, which represents the number of movies.

hiddenUnits = 20 #broj na skrieni nevroni

#The bias for the visible units is initialized to a tensor of zeros with a length equal to the number of visible units.
vb = tf.Variable(tf.zeros([visibleUnits]), tf.float32) #visibleUnitsBias

hb = tf.Variable(tf.zeros([hiddenUnits]), tf.float32) #hiddenUnitsBias. We also store the data type

w = tf.Variable(tf.zeros([visibleUnits, hiddenUnits]), tf.float32) # #The weight matrix between the visible and hidden units is initialized to a tensor of zeros with dimensions

Variable is a special kind of tensor that represents a modifiable tensor. It is used to store and update parameters in a model during training. Unlike regular tensors, which are immutable, Variables can be changed using operations such as assign and assign_add.


Следно, ги креираме влезниот (видливиот) и скриениот слој и специфицираме која активациска функција ќе се користи.

Дефинираме функција hidden_layer за креирање на скриениот слој.

1.v0_state - state of the visible units, W (weight matrix between the visible and hidden units), hb - hidden bias (the bias for the hidden units)

2.computes the probabilities of the hidden units being activated using the sigmoid function . tf.matmul([v0_state], W) + hb performs a matrix multiplication between the visible state and the weight matrix, then adds the hidden biases.

3.#sample_h_given_X. Samples the state of the hidden units based on the computed probabilities.



In [63]:
def hidden_layer(v0_state, W, hb): 
    
    h0_prob = tf.nn.sigmoid(tf.matmul([v0_state], W) + hb)  
    
    h0_state = tf.nn.relu(tf.sign(h0_prob - tf.random.uniform(tf.shape(h0_prob)))) 
    
    return h0_state

Дефинираме функција reconstructed_output за реконструкција на сигналот.


In [66]:
def reconstructed_output(h0_state, W, vb):
        
    v1_prob = tf.nn.sigmoid(tf.matmul(h0_state, tf.transpose(W)) + vb) # the prob of the visible units using the sigmoid function. 
    
    v1_state = tf.nn.relu(tf.sign(v1_prob - tf.random.uniform(tf.shape(v1_prob)))) #sample_v_given_h
    
    return v1_state[0]

Потоа ја креираме невронската мрежа.

In [68]:
v0 = tf.zeros([visibleUnits], tf.float32)

h0 = hidden_layer(v0, w, hb)

v1 = reconstructed_output(h0, w, vb)

Со следниве наредби може да го погледнеме бројот на неврони во секој слој.

In [69]:
print("hidden state shape: ", h0.shape)
print("v0 state shape:  ", v0.shape)
print("v1 state shape:  ", v1.shape)

hidden state shape:  (1, 20)
v0 state shape:   (3706,)
v1 state shape:   (3706,)


Дефинираме функција error за процена на грешката.

In [70]:
def error(v0_state, v1_state):
    return tf.reduce_mean(tf.square(v0_state - v1_state))

err = tf.reduce_mean(tf.square(v0 - v1))

# Tренирање на мрежата

Мрежата нека се тренира 5 епохи. Во примерот се користат batch-ови со големина 500 со што се добиваат 12 batch-ови.

In [71]:
epochs = 5
batchsize = 500
errors = []
weights = [] # the weights learned during training
K=1
alpha = 0.1 # learning rate  - kontrolira kolku mnogu tezinite ke se azuriraat na sekoj cekor

train_ds = tf.data.Dataset.from_tensor_slices((np.float32(trX))).batch(batchsize) #trX - dataset, kade sto from_tensor_slices go konvertira dataset-ot vo TensorFlow dataset. batch(batchsize) go deli datasetot vo poveek batches 

v0_state=v0 #initial state of the visible layer 
for epoch in range(epochs): #iteriraj niz 5te epohi
    batch_number = 0 
    for batch_x in train_ds:

        #Contrastive Divergence Algorithm
        for i_sample in range(len(batch_x)): # Loop through each sample in batch
            for k in range(K): # Gibbs sampling steps
                
                # Forward pass - visible to hidden 
                v0_state = batch_x[i_sample] #set input sample. Take a single training example
                h0_state = hidden_layer(v0_state, w, hb) # compute hidden layer activations using w and hb
                
                # Reconstruction - hidden to visible 
                v1_state = reconstructed_output(h0_state, w, vb) #reconstrict visible layer
                h1_state = hidden_layer(v1_state, w, hb) #reconstruct hidden layer

                # update weights 
                delta_W = tf.matmul(tf.transpose([v0_state]), h0_state) - tf.matmul(tf.transpose([v1_state]), h1_state)
                w = w + alpha * delta_W #update the weights using the learning rate alpha 
                
                vb = vb + alpha * tf.reduce_mean(v0_state - v1_state, 0) #update the visible bias
                hb = hb + alpha * tf.reduce_mean(h0_state - h1_state, 0) #update the hidden bias

                v0_state = v1_state

            if i_sample == len(batch_x)-1:
                err = error(batch_x[i_sample], v1_state)
                
                errors.append(err)
                weights.append(w)
                
        batch_number += 1

# Препорачување на филмови

Следно, може да се препорачуваат филмови за даден корисник (се специфицира неговото ID).

In [73]:
rec_user_id = 100

На влез се даваат преференците на корисникот (филмови кои ги гледал тој корисник), со што со мрежата треба да се реконструира влезот односно да се процени кои филмови би му се допаднале на корисникот врз база на неговите преференци.

In [75]:
inputUser = trX[rec_user_id-1].reshape(1, -1) #trX e datasetot koj sto ima normalizirani user ratings , i potoa se zima ID-to (indeksirano so 0, zatoa -1) i se pravi reshape vo 2D array so 1 redica i bilo kolku koloni sto se potrebni(-1) 

inputUser = tf.convert_to_tensor(trX[rec_user_id-1],"float32") #gi konvertira podatocite na korisnikot vo TensorFlow tensor

v0 = inputUser #go dodeluva tensorot na v0 koja e sostojbata na (visible layer) vo RBM

Врз база на влезниот вектор (преференците на корисникот) правиме реконструкција на влезот.


In [76]:
hh0 = tf.nn.sigmoid(tf.matmul([v0], w) + hb) #mnozam vlezot so tezinite i go dodavam BIAS-ot. Ja stavam sigmoid funkcijata za da gi dobijam verojatnostite na skrienite nevroni da bidat aktivirani

vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(w)), vb) #performs matrix multiplication between the hidden layer probabilities and the transposed weight matrix

rec = vv1

tf.maximum(rec, 1)

<tf.Tensor: shape=(1, 3706), dtype=float32, numpy=array([[1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

Transposing is the process of swapping the rows and columns of a matrix. In other words, the element at position (i, j) in the original matrix will be at position (j, i) in the transposed matrix. This operation is commonly used in linear algebra and various machine learning algorithms.

Transposing the matrix is necessary in this context to align the dimensions correctly for matrix multiplication. When reconstructing the visible layer from the hidden layer, the weight matrix w needs to be transposed to match the dimensions of the hidden layer activations

Реконструираниот вектор може да го погледнеме со:

In [77]:
for i in vv1:
    print(i)

tf.Tensor([0.9070024  0.6557478  0.04407573 ... 0.00811828 0.00170686 0.00728074], shape=(3706,), dtype=float32)


Потоа може да ги вратиме првите N (во случајов 10) највисоко рангирани филмови за тој корисник.

In [78]:
scored_movies_df_rec = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)] #filters the movies to include only those that have been rated by users.

scored_movies_df_rec = scored_movies_df_rec.assign(RecommendationScore = rec[0]) #adds a new column RecommendationScore to the DataFrame with the recommendation scores

scored_movies_df_rec.sort_values(["RecommendationScore"], ascending=False).head(10)

,MovieID,Movie,Genre,RecommendationScore
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,0.989046
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,0.976899
1178,1196,Star Wars: Episode V - The Empire Strikes Back (1980),Action|Adventure|Drama|Sci-Fi|War,0.976020
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,0.970713
770,780,Independence Day (ID4) (1996),Action|Sci-Fi|War,0.967154
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi,0.966778
1271,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure,0.960294
1023,1036,Die Hard (1988),Action|Thriller,0.960103
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller,0.959756
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi,0.959653
